# Board PL DDR4
----

## Aim/s
* Use `pynq` MMIO class to probe the PL DDR4 memory controller in the base overlay.

## References
* [PYNQ MMIO class](https://pynq.readthedocs.io/en/latest/pynq_libraries/mmio.html?highlight=MMIO)

## Last revised
* 27Jan21 - Initial revision
* 28Apr23 - Random Memory Testing
----

## Probe DDR4 memory controller

In [ ]:
import pynq
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [ ]:
print("Physical Address: ", hex(base.mem_dict['ddr4_0']['phys_addr']))
print("Address Range   : ", hex(base.mem_dict['ddr4_0']['addr_range']))

In [ ]:
from pynq import MMIO
import numpy as np
baseAddress = base.mem_dict['ddr4_0']['phys_addr']
mem_range = base.mem_dict['ddr4_0']['addr_range']
midpoint = mem_range // 2
mmioLO = MMIO(baseAddress, midpoint)
dramShadowLMB = mmioLO.array[0:mmioLO.length].view(np.int32)
mmioHI = MMIO(baseAddress + midpoint, midpoint)
dramShadowHMB = mmioHI.array[0:mmioHI.length].view(np.int32)

print("Partition PL-DRAM into upper and lower memory blocks")
print("LMB start: ", format(baseAddress, '016x'), "\tend: ", format(midpoint-1, '016x'))
print("HMB start: ", format(baseAddress+midpoint, '016x'), "\tend: ", format(baseAddress + 2*midpoint-1,'016x'))

The PL-DRAM is partitioned into two blocks, low (LMB) and high (HMB) memory.

# Random Memory Testing
We next generate an array of random values to store into memory using MMIO.  The PS creates a random block of values _numSamples_ long in its memory and then copies them to the PL DRAM.  A CRC32 is calculated and compared.  If a mismatch is found, the index of the first difference is shown.

The test can be customized by changing the number of samples (_numSamples_) and where in the overall PL-DRAM address map the samples are copied to to using the _dStart_ index.  The number of samples should be sufficiently long enough to exceed the processor caches but not too large otherwise run-time will be adversely effected.

In [ ]:
# Create the Source Buffer that will reside in PS memory space
numSamples = 64 * (1<<20) #  256 MB
srcBufferLMB = np.random.randint(np.iinfo(np.int32).max+1, size=numSamples, dtype=np.int32)
srcBufferHMB = np.random.randint(np.iinfo(np.int32).max+1, size=numSamples, dtype=np.int32)

In [ ]:
# Create set of start addresses to write to in PL DRAM
dStart = [0] * 4
dStart[0] = 0                           # start
dStart[1] = midpoint // (4 * 4)         # middle lower
dStart[2] = midpoint // (4 * 2)         # middle upper
dStart[3] = midpoint // 4 - numSamples  # end

In [ ]:
# Verify PL DRAM by writing to it and then reading back
for dramShadow, srcBuffer, memory_type in [(dramShadowLMB, srcBufferLMB, "Lower"), (dramShadowHMB, srcBufferHMB, "Upper")]:
    mismatch = 0
    for N in range(len(dStart)):
        dramShadow[dStart[N]:dStart[N]+len(srcBuffer)] = srcBuffer
        if np.array_equal(srcBuffer, dramShadow[dStart[N]:dStart[N]+len(srcBuffer)]):
            print(".", end='')
        else:
            index = np.argmax(srcBuffer != dramShadow[dStart[N]:dStart[N]+len(srcBuffer)])
            print(f"\n{memory_type} Memory Test Failed: not matching starting at index: {index}")
            print(f"\t\t{srcBuffer[index]} :: {dramShadow[dStart[N]+index]}")
            mismatch += 1
            break
    if (mismatch == 0):
        print(f"\n{memory_type} Block Random Memory Test Verified")

---
Copyright (C) 2023 Advanced Micro Devices, Inc
SPDX-License-Identifier: MIT
---